inladen van alle foto id's in een dataframe met bijbehorende label uit map naam <br>
predictions van de ensemble methode op val_gen<br>
deze predicties als library in een dataframe toevoegen <br>
Dan for loop checken welke nummers niet kloppen

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.metrics import roc_curve, auc

map0 = r"C:\Users\20192735\Documents\8P361_images\train+val\valid\0"
map1 = r"C:\Users\20192735\Documents\8P361_images\train+val\valid\1"
filenames0 = os.listdir(map0)
filenames1 = os.listdir(map1)
zeros = [0]*len(filenames0)
ones = [1]*len(filenames1)

df_true_0 = pd.DataFrame(filenames0, columns=["filename"])
df_true_0["true_label"]=zeros
df_true_1 = pd.DataFrame(filenames1, columns=["filename"])
df_true_1["true_label"]=ones

df_true = pd.concat([df_true_0,df_true_1])
df_true= df_true.set_index("filename")

df_true.head()


,true_label
filename,
0006ed0cc8fa6edb4d2a74094cf2d26c85a1a33c.jpg,0
0007da302a322ef9444dc55227536cbb61e18781.jpg,0
0033eb6f1ae535166311e5837b006cba336e9200.jpg,0
003ff6ef7139d48052a5f60adb08304325765a9f.jpg,0
0043f20e46038846be8aea6cbacf75f7f17716ee.jpg,0


In [2]:
df_inception_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\Inception_valid.csv')
df_efficient_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\EfficientNet_valid.csv')
df_resnet_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\ResNet_valid.csv')
df_mobilenet_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\MobileNetV2_valid.csv')
df_xception_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\Xception_valid.csv')

df_train_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\train_valid.csv')
df_averagingdy_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\averaging_dynamic_valid.csv')
df_majorityex_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\majority_extremes_valid.csv')
df_averaging_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\averaging_valid.csv')
df_majority_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\majority_valid.csv')
df_bayes_old = pd.read_csv (r'C:\Users\20192735\OneDrive - TU Eindhoven\8P361 - Project Imaging\8p361-project-imaging-master\assignments\Good data\bayes_valid.csv')


df_majority = df_majority_old.rename(columns={'label':'label_majority'}, errors='raise')
df_averaging = df_averaging_old.rename(columns={'label':'label_averaging'}, errors='raise')
df_majorityex = df_majorityex_old.rename(columns={'label':'label_majorityex'}, errors='raise')
df_averagingdy = df_averagingdy_old.rename(columns={'label':'label_averagingdy'}, errors='raise')
df_bayes = df_bayes_old.rename(columns={'label':'label_bayes'}, errors='raise')
df_train = df_train_old.rename(columns={'label':'label_train'}, errors='raise')

df_inception = df_inception_old.rename(columns={'label':'label_inception'}, errors='raise')
df_efficient = df_efficient_old.rename(columns={'label':'label_efficient'}, errors='raise')
df_resnet = df_resnet_old.rename(columns={'label':'label_resnet'}, errors='raise')
df_mobilenet = df_mobilenet_old.rename(columns={'label':'label_mobilenet'}, errors='raise')
df_xception = df_xception_old.rename(columns={'label':'label_xception'}, errors='raise')


all_probs = pd.DataFrame(data=[df_efficient['id'],df_train["label_train"],df_averaging["label_averaging"],df_averagingdy["label_averagingdy"],df_bayes["label_bayes"] ,df_majorityex["label_majorityex"],df_majority["label_majority"],df_efficient['label_efficient'], df_inception['label_inception'], df_xception['label_xception'], df_resnet['label_resnet'], df_mobilenet['label_mobilenet']])
all_probs = all_probs.transpose()
all_probs["filename"]= all_probs["id"]+".jpg"
all_probs= all_probs.set_index("filename")
all_probs.head()


,id,label_train,label_averaging,label_averagingdy,label_bayes,label_majorityex,label_majority,label_efficient,label_inception,label_xception,label_resnet,label_mobilenet
filename,,,,,,,,,,,,
00001b2b5609af42ab0ab276dd4cd41c3e7745b5.jpg,00001b2b5609af42ab0ab276dd4cd41c3e7745b5,1.031553,0.993553,0.993626,0.989513,0.999984,0.993553,0.999815,0.989513,0.978484,0.999967,0.999984
00039ed67c14391535d5ac933cc4854619849738.jpg,00039ed67c14391535d5ac933cc4854619849738,1.03447,0.996346,0.996367,0.99771,0.99995,0.996346,0.998477,0.99771,0.998101,0.99995,0.987494
0006ed0cc8fa6edb4d2a74094cf2d26c85a1a33c.jpg,0006ed0cc8fa6edb4d2a74094cf2d26c85a1a33c,0.0,0.016084,0.01593,0.017163,0.000022,0.016084,0.017163,0.032629,0.004706,0.025901,0.000022
0007da302a322ef9444dc55227536cbb61e18781.jpg,0007da302a322ef9444dc55227536cbb61e18781,0.0,0.011072,0.010765,0.005513,0.000201,0.011072,0.000686,0.003256,0.005513,0.045705,0.000201
0007e4f8f2e61aa10a7cdb647e8ed346a9b4398a.jpg,0007e4f8f2e61aa10a7cdb647e8ed346a9b4398a,0.394053,0.391899,0.371564,0.21952,0.096537,0.175294,0.209824,0.21952,0.096537,0.715984,0.717632


In [3]:
true_and_pred = df_true.join(all_probs)
true_and_pred.drop('id', inplace=True, axis=1)

true_and_pred= true_and_pred.astype("float").round(0).astype(int)

true_and_pred.head()
#print(true_and_pred["label_efficient"][1])

,true_label,label_train,label_averaging,label_averagingdy,label_bayes,label_majorityex,label_majority,label_efficient,label_inception,label_xception,label_resnet,label_mobilenet
filename,,,,,,,,,,,,
0006ed0cc8fa6edb4d2a74094cf2d26c85a1a33c.jpg,0,0,0,0,0,0,0,0,0,0,0,0
0007da302a322ef9444dc55227536cbb61e18781.jpg,0,0,0,0,0,0,0,0,0,0,0,0
0033eb6f1ae535166311e5837b006cba336e9200.jpg,0,0,0,0,0,0,0,0,0,0,0,0
003ff6ef7139d48052a5f60adb08304325765a9f.jpg,0,0,0,0,0,0,0,0,0,0,0,0
0043f20e46038846be8aea6cbacf75f7f17716ee.jpg,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
y_true = true_and_pred["true_label"]
y_pred = true_and_pred["label_efficient"]


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
# TP , FP
# FN , TN
FN_A = cm[1][0]
TP_A = cm[0][0]
FNR_A = FN_A/(FN_A+ TP_A)
print(FNR_A)

0.07900899790459756


In [62]:
test_df = pd.DataFrame({'filename': ["a","b","c","d","e","f"],
                   'true_label': [0,0,0,1,1,1], "ensemble_A" : [1,1,1,1,1,1], "ensemble_B" : [1,0,1,0,1,1]})
test_df=test_df.set_index("filename")
test_df.head()


y_true = test_df["true_label"]
y_pred = test_df["ensemble_A"]

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
# TP , FP
# FN , TN
FP_A = cm[0][1]
TN_A = cm[1][1]
FPR_A = FP_A/(FP_A+ TN_A)
print(FPR_A)

0.5
